In [3]:
import pandas as pd
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib as mpl
from matplotlib import cm as cm
import math
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns



global dataFrame, dfreg
global moving_avg
global dfcomp
global clfknn
global clfknndist
global X, y, X_train, y_train, X_test, y_test,X_pred
global distknn, uniknn, knnunipred, knndistpred


def loadDataset():
    global dataFrame
    global dfcomp
    start = datetime.datetime(2010, 1, 1)
    end = datetime.datetime(2020, 1, 1)

    dataFrame = web.DataReader("AAPL", 'yahoo', start, end)
    print("Shape of Apple Stock Dataset: "+str(dataFrame.shape)+"\n\n")
    print("Sample of Apple Stock Data: \n"+str(dataFrame.head(2))+"\n\n")

    dfcomp = web.DataReader(['AAPL', 'GE', 'GOOG', 'IBM', 'MSFT'], 'yahoo', start=start, end=end)['Adj Close']
    #dfcomp = pd.read_csv("/content/sample_data/competitor_YahooDataSet.csv")['Adj Close']
    dfcomp.rename(columns = {'AAPL':'TCS', 'GE':'INFOSYS', 'GOOG':'RELIANCE', 'MSFT':'BAJAJ'}, inplace=True)
    print("Shape of Apple Competitor Stock Dataset: " + str(dfcomp.shape) + "\n\n")
    print("Sample of Apple Competitor Stock Data: \n" + str(dfcomp.head(2)) + "\n\n")

    print("Dataset Downloaded from Yahoo Finance Dataset\n\n")

def dfcorr():
    global dfcomp

    print("Correlation table for Apple Competitor Stock\n\n")
    retscomp = dfcomp.pct_change()
    corr = retscomp.corr()
    print("correlation: \n"+str(corr)+"\n\n")

def dataPreProcess():
    global dataFrame,dfreg
    global X, y, X_train, X_test, y_train, y_test,X_pred

    print("Data PreProcessing for Apple Stock Dataset\n\n")
    dfreg = dataFrame.loc[:,["Adj Close","Volume"]]
    dfreg["HL_PCT"] = (dataFrame["High"] - dataFrame["Low"]) / dataFrame["Close"] * 100.0
    dfreg["PCT_change"] = (dataFrame["Close"] - dataFrame["Open"]) / dataFrame["Open"] * 100.0

    # Drop missing value
    dfreg.fillna(value=-99999, inplace=True)
    # We want to separate 1 percent of the data to forecast
    forecast_out = int(math.ceil(0.01 * len(dfreg)))

    # Separating the label here, we want to predict the AdjClose
    forecast_col = 'Adj Close'
    dfreg['label'] = dfreg[forecast_col].shift(-forecast_out)
    X = np.array(dfreg.drop(['label'], 1))

    # Scale the X so that everyone can have the same distribution for linear regression
    X = preprocessing.scale(X)
    # Finally We want to find Data Series of late X and early X (train) for model generation and evaluation
    X_pred = X[-forecast_out:]
    X = X[:-forecast_out]
    # Separate label and identify it as y
    y = np.array(dfreg['label'])
    y = y[:-forecast_out]
    

    print("X lablels : \n"+str(X)+"\n\n")
    print("Y lablels : \n"+str(y)+"\n\n")
    print("Data spliting into Train and Test")
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

    print("number of Train Samples : " + str(len(X_train)) + "\n")
    print("number of Test Sample: " + str(len(X_test)) + "\n")

    print("Data Preprocessing Completed\n\n")

def uniformKNN():
    global clfknn
    global uniknn

    # KNN Regression
    clfknn = KNeighborsRegressor(n_neighbors=5)
    clfknn.fit(X_train, y_train)

    uniknn = clfknn.score(X_train, y_train)

    print("Accuracy of KNN with Uniform weights : "+str(uniknn*100)+"\n\n")

def distKNN():
    global clfknndist,knndistpred
    global distknn,knnunipred
    # KNN Regression
    clfknndist = KNeighborsRegressor(n_neighbors=5,weights='distance')
    clfknndist.fit(X_train, y_train)
    distknn = clfknndist.score(X_train, y_train)

    print("Accuracy of KNN with Distinct weights : "+str(distknn*100)+"\n\n")
def predModel():
  
    global clfknndist,clfknn,knnunipred,knndistpred
    global X, y, X_train, X_test, y_train, y_test
    test = pd.read_csv("/content/sample_data/pred.csv")
    print(" test file loaded\n"+str(test.columns)+"\n");
    x_pred = np.array(test.drop(['Unnamed: 0'],1))

    print("test Dataset: \n"+str(x_pred)+"\n\n");

    knndistpred = clfknndist.predict(x_pred)

    print("Predict values for KNN with Dist weights: \n" + str(knndistpred) + "\n\n");

    knnunipred = clfknn.predict(x_pred)

    print("Predict values for KNN with Uni Wights: \n" + str(knnunipred) + "\n\n");



#loading dataset
loadDataset()
#Finding the correlation between the features
dfcorr()
#Preprocessing the data
dataPreProcess()
#KNN with uniform weights
uniformKNN()
#KNN with distinct weights
distKNN()


#Working on these two functions
predModel()
#graph()

Shape of Apple Stock Dataset: (2516, 6)


Sample of Apple Stock Data: 
                 High        Low       Open      Close       Volume  Adj Close
Date                                                                          
2010-01-04  30.642857  30.340000  30.490000  30.572857  123432400.0  26.466835
2010-01-05  30.798571  30.464285  30.657143  30.625713  150476200.0  26.512596


Shape of Apple Competitor Stock Dataset: (2516, 5)


Sample of Apple Competitor Stock Data: 
Symbols           TCS    INFOSYS    RELIANCE        IBM      BAJAJ
Date                                                              
2010-01-04  26.466835  10.749147  312.204773  94.724556  24.294369
2010-01-05  26.512596  10.804806  310.829926  93.580284  24.302216


Dataset Downloaded from Yahoo Finance Dataset


Correlation table for Apple Competitor Stock


correlation: 
Symbols        TCS   INFOSYS  RELIANCE       IBM     BAJAJ
Symbols                                                   
TCS       1.000000  0

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:173: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


TypeError: ignored

In [5]:
len(X_pred)

26

In [0]:
# importing the required module 
import matplotlib.pyplot as plt 

# x axis values 
x = [1,2,3] 
# corresponding y axis values 
y = [2,4,1] 

# plotting the points 
plt.plot(x, y) 

# naming the x axis 
plt.xlabel('x - axis') 
# naming the y axis 
plt.ylabel('y - axis') 

# giving a title to my graph 
plt.title('My first graph!') 

# function to show the plot 
plt.show() 


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fa798c55158> (for post_execute):


ValueError: ignored